In [ ]:
from itkwidgets import view
from ipywidgets import widgets, interactive
import numpy as np
from vtkmodules.vtkIOXML import vtkXMLPolyDataReader
from vtkmodules.vtkFiltersPoints import *
from vtk import vtkIdList, vtkOutlineFilter, VTK_POLY_VERTEX
# Tested Fri 14 May 2021 04:28:25 PM CEST

# Read a point cloud from disk

In [ ]:
planet = vtkXMLPolyDataReader()
planet.SetFileName("/scratch/snx3000/jfavre/PlanetsForming/PlanetsForming_0_0.vtp")
planet.SetPointArrayStatus("Mass",0)
planet.SetPointArrayStatus("rho",1)
planet.SetPointArrayStatus("hsmooth",0)
planet.Update()
planet.GetOutput().GetPointData().SetActiveScalars("rho")
print("Planet has", planet.GetOutput().GetNumberOfPoints(), "vertices")

In [ ]:
hBin = vtkHierarchicalBinningFilter()
hBin.SetInputConnection(planet.GetOutputPort())
#hBin.AutomaticOn()
hBin.AutomaticOff()
hBin.SetDivisions(2,2,2)
hBin.SetBounds(planet.GetOutput().GetBounds())
hBin.Update()
print(hBin.GetNumberOfLevels())
print(hBin.GetNumberOfGlobalBins())
for i in range(hBin.GetNumberOfLevels()):
    print(hBin.GetNumberOfBins(i))

In [ ]:

extBin = vtkExtractHierarchicalBins()
extBin.SetInputConnection(hBin.GetOutputPort())
extBin.SetBinningFilter(hBin)
extBin.SetLevel(0)
extBin.SetBin(0)
extBin.Update()
nnodes = extBin.GetOutput().GetNumberOfPoints()
ptIds = vtkIdList()
ptIds.SetNumberOfIds(nnodes)
for a in range(nnodes):
  ptIds.SetId(a, a)
extBin.GetOutput().Allocate(1)
extBin.GetOutput().InsertNextCell(VTK_POLY_VERTEX , ptIds)
print(extBin.GetOutput().GetNumberOfPoints())

In [ ]:
# Create an outline
outline = vtkOutlineFilter()
outline.SetInputConnection(planet.GetOutputPort())
outline.Update()

In [ ]:
viewer = view(geometries=[], point_sets=[extBin.GetOutput()])

def SelectLevel(l):
    extBin.SetLevel(l)
    extBin.SetBin(-1)
    extBin.Update()
    nnodes = extBin.GetOutput().GetNumberOfPoints()
    ptIds = vtkIdList()
    ptIds.SetNumberOfIds(nnodes)
    for a in range(nnodes):
      ptIds.SetId(a, a)
    extBin.GetOutput().Allocate(1)
    extBin.GetOutput().InsertNextCell(VTK_POLY_VERTEX , ptIds)
    viewer.point_sets=[extBin.GetOutput()]
    
slider = interactive(SelectLevel, l=(0,1))
widgets.VBox([viewer, slider])